### Instalação e carregamento das bibliotecas de GA em python

In [ ]:
#Executar no prompt do Anaconda: pip install deap

In [1]:
import random
import numpy
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
import math
import pandas as pd

### É necessário fazer o upload dos seguintes arquivos de entrada:
- Portos
- Embarcações
- Plataformas
- Distâncias entre os Portos onde estão as Embarcações e as Plataformas

In [2]:
Portos = pd.read_excel('TCC.xlsx', sheet_name = 'Portos', nrows = 10, header = 0, index_col = 0)
Portos

,Latitude,Longitude
Portos,,
PORTO DO ACU,-21.843250,-41.00089
ANGRA DOS REIS,-23.011830,-44.31065
CABEDELO,-6.971611,-34.83998
FORTALEZA,-3.714480,-38.49648
TMIB,-10.845280,-36.91853
PECEM,-3.532126,-38.80311
RIO DE JANEIRO,-22.849820,-43.17784
SANTOS,-23.951090,-46.35358
SAO SEBASTIAO,-23.795470,-45.39520


In [3]:
Embarcacoes = pd.read_excel('TCC.xlsx', sheet_name = 'Embarcacoes', nrows = 10, header = 0, index_col = 0)
Embarcacoes

,Velocidade,Consumo,Localização
Embarcações,,,
Embarcação 01,10,80,ANGRA DOS REIS
Embarcação 02,11,60,TMIB
Embarcação 03,12,70,CABEDELO
Embarcação 04,13,80,PORTO DO ACU
Embarcação 05,14,60,SUAPE
Embarcação 06,15,70,RIO DE JANEIRO
Embarcação 07,16,80,FORTALEZA
Embarcação 08,17,60,PECEM
Embarcação 09,18,70,SANTOS


In [4]:
Plataformas = pd.read_excel('TCC.xlsx', sheet_name = 'Plataformas', nrows = 10, header = 0, index_col = 0)
Plataformas

,Latitude,Longitude
Plataformas,,
Plataforma 01,"-24,109335","-39,813312"
Plataforma 02,"-31,36303","-40,61051"
Plataforma 03,"-26,244168","-45,708166"
Plataforma 04,"-24,75682","-47,015539"
Plataforma 05,"-24,468399","-43,202861"
Plataforma 06,"-23,812492","-43,288848"
Plataforma 07,"-16,207462","-33,628008"
Plataforma 08,"-10,977465","-28,767821"
Plataforma 09,"-3,335002","-28,780897"


In [5]:
Distancias = pd.read_excel('TCC.xlsx', sheet_name = 'Distancias', nrows = 10, header = 0, index_col = 0)
Distancias

,Plataforma 01,Plataforma 02,Plataforma 03,Plataforma 04,Plataforma 05,Plataforma 06,Plataforma 07,Plataforma 08,Plataforma 09,Plataforma 10
De \ Para,,,,,,,,,,
PORTO DO ACU,150.612545,977.064371,1059.518210,1121.096441,912.149089,916.784818,537.494945,1483.601471,1944.382935,474.894890
ANGRA DOS REIS,256.447291,820.978511,207.869515,932.262055,106.331802,73.975469,728.594346,1745.563359,2209.638501,638.256237
CABEDELO,1995.126697,2427.757573,2428.908835,2423.192830,2200.067787,2169.885908,982.982207,432.624769,422.708264,1613.154080
FORTALEZA,1991.560281,2437.175870,2429.059185,2420.923238,2197.174676,2165.961047,1075.183512,789.685337,583.320893,1609.553312
TMIB,1306.401913,2085.097078,2080.982366,1632.368033,1454.677696,1433.482471,373.592426,481.171283,1000.746672,1124.779758
PECEM,1985.726625,2432.428489,2423.533617,2415.194819,2191.395863,2160.095929,1082.721789,808.012376,601.450510,1603.719776
RIO DE JANEIRO,200.280642,858.200325,926.832414,994.674854,96.804169,57.892777,670.825127,1669.484587,2132.431132,577.854486
SANTOS,359.340823,730.534047,141.589026,60.312052,175.566644,168.735019,1064.898101,1919.581235,2385.695974,744.445936
SAO SEBASTIAO,307.343554,753.506774,147.453103,105.814109,126.877088,115.904333,1032.811545,1854.177509,2319.334690,692.530144


### Declaração das funções auxiliares que devem conter as informações do problema

In [6]:
#Função para gerar a tabela para o Power BI
def geraDadosPBICenarioInicial():
    ordem = [4, 2, 10, 9, 1, 6, 8, 3, 7, 5]
    
    dfPBI = pd.DataFrame(columns = ['ID', 'Ordem', 'Ponto', 'Latitude', 'Longitude', 'Emissão CO2'], index = None)
    for o in range(len(ordem)):
        if ordem[o] < 10:
            Emba = "Embarcação 0"
        else:
            Emba = "Embarcação "
        if o < 9:
            Plat = "Plataforma 0"
        else:
            Plat = "Plataforma "
        Porto = Embarcacoes.iloc[ordem[o]-1,2]
        Distancia = Distancias.loc[Porto][o]
        #Tempo = Distancia / Velocidade / 24 (horas por dia)
        Tempo = Distancia / Embarcacoes.iloc[ordem[o]-1,0] / 24
        #Emissao = Tempo * Consumo * 3.18 (transformação do consumo de combustível para emissão de CO2)
        Emissao = Tempo * Embarcacoes.iloc[ordem[o]-1,1] * 3.18
        contador = 1
        dfPBI = dfPBI.append(
            pd.DataFrame({
            'ID': [o+1],
            'Ordem': [contador],
            'Ponto': [Emba + str(ordem[o])],
            'Latitude': [str(Portos.loc[Porto][0]).replace('.',',')],
            'Longitude': [str(Portos.loc[Porto][1]).replace('.',',')],
            'Emissão CO2': ['']}),
            ignore_index = True)
        contador += 1
        if (abs(abs(float(str(Plataformas.loc[Plat + str(o+1)][0]).replace(',','.'))) - abs(float(str(Portos.loc[Porto][0]).replace(',','.')))) > 7) or ((abs(float(str(Plataformas.loc[Plat + str(o+1)][1]).replace(',','.'))) - abs(float(str(Portos.loc[Porto][1]).replace(',','.')))) > 2):
            dfPBI = dfPBI.append(
                pd.DataFrame({
                'ID': [o+1],
                'Ordem': [contador],
                'Ponto': [''],
                'Latitude': [str(float(str(Portos.loc[Porto][0]).replace(',','.')) - 2).replace('.',',')],
                'Longitude': [str(float(str(Portos.loc[Porto][1]).replace(',','.')) + 7).replace('.',',')],
                'Emissão CO2': ['']}),
                ignore_index = True)
            contador += 1
            if (abs(abs(float(str(Plataformas.loc[Plat + str(o+1)][0]).replace(',','.'))) - abs(float(str(Portos.loc[Porto][0]).replace(',','.')) - 2)) > 12):
                dfPBI = dfPBI.append(
                    pd.DataFrame({
                    'ID': [o+1],
                    'Ordem': [contador],
                    'Ponto': [''],
                    'Latitude': [Plataformas.loc[Plat + str(o+1)][0]],
                    'Longitude': [str(float(str(Portos.loc[Porto][1]).replace(',','.')) + 7).replace('.',',')],
                    'Emissão CO2': ['']}),
                    ignore_index = True)
                contador += 1
        dfPBI = dfPBI.append(
            pd.DataFrame({
            'ID': [o+1],
            'Ordem': [contador],
            'Ponto': [Plat + str(o+1)],
            'Latitude': [Plataformas.loc[Plat + str(o+1)][0]],
            'Longitude': [Plataformas.loc[Plat + str(o+1)][1]],
            'Emissão CO2': [str(Emissao).replace('.',',')]}),
            ignore_index = True)
    return dfPBI,

In [7]:
dfTabelaPBI = pd.DataFrame(columns = ['ID', 'Ordem', 'Ponto', 'Latitude', 'Longitude', 'Emissão CO2'])
dfTabelaPBI = dfTabelaPBI.append(geraDadosPBICenarioInicial(), ignore_index = True)
dfTabelaPBI

,ID,Ordem,Ponto,Latitude,Longitude,Emissão CO2
0,1,1,Embarcação 04,"-21,84325","-41,00089",
1,1,2,Plataforma 01,"-24,109335","-39,813312","122,80715181268616"
2,2,1,Embarcação 02,"-10,84528","-36,91853",
3,2,2,,"-12,84528","-29,918529999999997",
4,2,3,,"-31,36303","-29,918529999999997",
5,2,4,Plataforma 02,"-31,36303","-40,61051","1506,9565242592998"
6,3,1,Embarcação 10,"-23,79547","-45,3952",
7,3,2,Plataforma 03,"-26,244168","-45,708166","82,26331002098343"
8,4,1,Embarcação 09,"-23,95109","-46,35358",
9,4,2,Plataforma 04,"-24,75682","-47,015539","31,077460365943953"


In [8]:
dfTabelaPBI.to_csv("ResultadoCenarioInicial.csv", sep = ";", encoding = "cp1252")